# Introduction

This notebook demonstrates basic usage of BioThings Explorer, an engine for autonomously querying a distributed knowledge graph. BioThings Explorer can answer two classes of queries -- "EXPLAIN" and "PREDICT".  EXPLAIN queries are described in [EXPLAIN_demo.ipynb](EXPLAIN_demo.ipynb). Here, we describe PREDICT queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

PREDICT queries are designed to **predict plausible relationships between one entity and an entity class**.  For example, in this notebook, we explore the question:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*"What drugs might be used to treat COVID-19 through endocytosis regulation?"*

Related Article: https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(20)30304-4/fulltext

## Step 0: Load BioThings Explorer modules

First, install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).
**To experiment with an executable version of this notebook, [load it in Google Colaboratory](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/COVID_demo.ipynb).**

In [ ]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [1]:
from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
ht = Hint()


## Step 1: Find representation of "Endocytosis" in BTE

In this step, BioThings Explorer translates our query string "Endocytosis"  into BioThings objects, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [8]:
endocytosis = ht.query("Endocytosis")['BiologicalProcess'][0]
endocytosis

{'GO': 'GO:0006897',
 'name': 'endocytosis',
 'primary': {'identifier': 'GO',
  'cls': 'BiologicalProcess',
  'value': 'GO:0006897'},
 'display': 'GO(GO:0006897) name(endocytosis)',
 'type': 'BiologicalProcess'}

## Step 2: Find drugs that are associated with genes which invovled in Endocytosis

In this section, we find all paths in the knowledge graph that connect Endocytosis to any entity that is a chemical compound.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

The parameters for `FindConnection` are described below:


In [9]:
fc = FindConnection(input_obj=endocytosis, output_obj='ChemicalSubstance', intermediate_nodes=['Gene'])
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'endocytosis' and 'ChemicalSubstance'.                   Paths will have 1 intermediate node.

Intermediate node #1 will have these type constraints: Gene



========== QUERY #1 -- fetch all Gene entities linked to endocytosis ==========

==== Step #1: Query path planning ====

Because endocytosis is of type 'BiologicalProcess', BTE will query our meta-KG for APIs that can take 'BiologicalProcess' as input and 'Gene' as output

BTE found 1 apis:

API 1. cord_biological_process(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 1.1: https://biothings.ncats.io/cord_bp/query?fields=associated_with (POST -d q=GO:0006897&scopes=go)


==== Step #3: Output normalization ====

API 1.1 cord_biological_process: 247 hits

After id-to-object translation, BTE retrieved 247 unique objects.


========== QUERY #2.1 -- fe

API 8.185: https://automat.renci.org/chembio/gene/chemical_substance/NCBIGene:83696
API 7.28: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000142208&datasource=chembl&size=100&fields=drug
API 8.184: https://automat.renci.org/chembio/gene/chemical_substance/NCBIGene:9342
API 8.176: https://automat.renci.org/chembio/gene/chemical_substance/NCBIGene:115482700
API 10.25: https://automat.renci.org/cord19_scigraph_v2/gene/chemical_substance/NCBIGene:4023
API 10.17: https://automat.renci.org/cord19_scigraph_v2/gene/chemical_substance/NCBIGene:1268
API 10.26: https://automat.renci.org/cord19_scigraph_v2/gene/chemical_substance/NCBIGene:23581
API 1.226: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=MCAT
API 7.48: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000185880&datasource=chembl&size=100&fields=drug
API 7.34: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG0000010

API 7.194: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000140795&datasource=chembl&size=100&fields=drug
API 1.18: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=PCYT1B
API 7.203: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000107014&datasource=chembl&size=100&fields=drug
API 7.52: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000138798&datasource=chembl&size=100&fields=drug
API 1.14: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=DAPK3
API 7.199: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000197919&datasource=chembl&size=100&fields=drug
API 1.16: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=PRDX1
API 7.210: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000176697&datasource=chembl&size=100&fields=drug
API 1.5: http://dgidb.genome.wustl.edu/api/v2/int

API 1.25: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=RNASE1
API 7.232: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000213927&datasource=chembl&size=100&fields=drug
API 7.64: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000150768&datasource=chembl&size=100&fields=drug
API 1.9: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=SNAP29
API 9.171: https://automat.renci.org/hmdb/gene/chemical_substance/NCBIGene:6010
API 9.172: https://automat.renci.org/hmdb/gene/chemical_substance/NCBIGene:54106
API 7.233: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000100764&datasource=chembl&size=100&fields=drug
API 1.28: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=DLAT
API 9.173: https://automat.renci.org/hmdb/gene/chemical_substance/NCBIGene:3439
API 9.174: https://automat.renci.org/hmdb/gene/chemical_substance/NCBIGene:118460
API 1.36: http:

API 1.43: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=CNR1
API 7.159: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000100519&datasource=chembl&size=100&fields=drug
API 7.154: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000127666&datasource=chembl&size=100&fields=drug
API 7.5: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000181544&datasource=chembl&size=100&fields=drug
API 1.50: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=TNFSF14
API 9.196: https://automat.renci.org/hmdb/gene/chemical_substance/NCBIGene:4431
API 1.108 dgidb failed
API 7.1: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG00000070831&datasource=chembl&size=100&fields=drug
API 1.42: http://dgidb.genome.wustl.edu/api/v2/interactions.json?genes=FCGRT
API 7.165: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?target=ENSG

...

API 2.25 scibite: 1 hits
API 2.26 scibite: No hits
API 2.27 scibite: 5 hits
API 2.28 scibite: No hits
API 2.29 scibite: No hits
API 2.30 scibite: No hits
API 2.31 scibite: 2 hits
API 2.32 scibite: No hits
API 2.33 scibite: No hits
API 2.34 scibite: No hits
API 2.35 scibite: No hits
API 2.36 scibite: No hits
API 2.37 scibite: No hits
API 2.38 scibite: 14 hits
API 2.39 scibite: No hits
API 2.40 scibite: 1 hits
API 2.41 scibite: 1 hits
API 2.42 scibite: 25 hits
API 2.43 scibite: No hits
API 2.44 scibite: No hits
API 2.45 scibite: No hits
API 2.46 scibite: No hits
API 2.47 scibite: No hits
API 2.48 scibite: No hits
API 2.49 scibite: 1 hits
API 2.50 scibite: No hits
API 2.51 scibite: 19 hits
API 2.52 scibite: No hits
API 2.53 scibite: No hits
API 2.54 scibite: No hits
API 2.55 scibite: No hits
API 2.56 scibite: No hits
API 2.57 scibite: No hits
API 2.58 scibite: No hits
API 2.59 scibite: No hits
API 2.60 scibite: No hits
API 2.61 scibite: No hits
API 2.62 scibite: 15 hits
API 2.63 scibite: 

API 10.106 scigraph: 38 hits
API 10.107 scigraph: No hits
API 10.108 scigraph: No hits
API 10.109 scigraph: No hits
API 10.110 scigraph: No hits
API 10.111 scigraph: No hits
API 10.112 scigraph: 1 hits
API 10.113 scigraph: 37 hits
API 10.114 scigraph: 38 hits
API 10.115 scigraph: No hits
API 10.116 scigraph: No hits
API 10.117 scigraph: 3 hits
API 10.118 scigraph: 2 hits
API 10.119 scigraph: 23 hits
API 10.120 scigraph: 37 hits
API 10.121 scigraph: 39 hits
API 10.122 scigraph: No hits
API 10.123 scigraph: No hits
API 10.124 scigraph: 1 hits
API 10.125 scigraph: 11 hits
API 10.126 scigraph: No hits
API 10.127 scigraph: No hits
API 10.128 scigraph: No hits
API 10.129 scigraph: No hits
API 10.130 scigraph: 176 hits
API 10.131 scigraph: No hits
API 10.132 scigraph: No hits
API 10.133 scigraph: No hits
API 10.134 scigraph: 3 hits
API 10.135 scigraph: No hits
API 10.136 scigraph: No hits
API 10.137 scigraph: No hits
API 10.138 scigraph: No hits
API 10.139 scigraph: No hits
API 10.140 scigrap

API 1.192 dgidb: 1 hits
API 1.193 dgidb: 19 hits
API 1.194 dgidb: No hits
API 1.195 dgidb: 2 hits
API 1.196 dgidb: No hits
API 1.197 dgidb: No hits
API 1.198 dgidb: No hits
API 1.199 dgidb: 72 hits
API 1.200 dgidb: 7 hits
API 1.201 dgidb: 4 hits
API 1.202 dgidb: No hits
API 1.203 dgidb: No hits
API 1.204 dgidb: 2 hits
API 1.205 dgidb: No hits
API 1.206 dgidb: 43 hits
API 1.207 dgidb: No hits
API 1.208 dgidb: No hits
API 1.209 dgidb: 3 hits
API 1.210 dgidb: No hits
API 1.211 dgidb: 5 hits
API 1.212 dgidb: No hits
API 1.213 dgidb: 2 hits
API 1.214 dgidb: 22 hits
API 1.215 dgidb: No hits
API 1.216 dgidb: No hits
API 1.217 dgidb: No hits
API 1.218 dgidb: No hits
API 1.219 dgidb: 21 hits
API 1.220 dgidb: No hits
API 1.221 dgidb: 2 hits
API 1.222 dgidb: No hits
API 1.223 dgidb: No hits
API 1.224 dgidb: No hits
API 1.225 dgidb: 4 hits
API 1.226 dgidb: No hits
API 1.227 dgidb: No hits
API 1.228 dgidb: 1 hits
API 1.229 dgidb: No hits
API 1.230 dgidb: 5 hits
API 1.231 dgidb: No hits
API 1.232 dg

API 8.56 chembio: 2 hits
API 8.57 chembio: 1 hits
API 8.58 chembio: No hits
API 8.59 chembio: 3 hits
API 8.60 chembio: 1 hits
API 8.61 chembio: No hits
API 8.62 chembio: 42 hits
API 8.63 chembio: No hits
API 8.64 chembio: No hits
API 8.65 chembio: No hits
API 8.66 chembio: 1 hits
API 8.67 chembio: 1 hits
API 8.68 chembio: 15 hits
API 8.69 chembio: 1 hits
API 8.70 chembio: 6 hits
API 8.71 chembio: No hits
API 8.72 chembio: No hits
API 8.73 chembio: No hits
API 8.74 chembio: 2 hits
API 8.75 chembio: No hits
API 8.76 chembio: No hits
API 8.77 chembio: 4 hits
API 8.78 chembio: 22 hits
API 8.79 chembio: 1 hits
API 8.80 chembio: No hits
API 8.81 chembio: 1 hits
API 8.82 chembio: 3 hits
API 8.83 chembio: 1 hits
API 8.84 chembio: 4 hits
API 8.85 chembio: No hits
API 8.86 chembio: 9 hits
API 8.87 chembio: 3 hits
API 8.88 chembio: 1 hits
API 8.89 chembio: No hits
API 8.90 chembio: No hits
API 8.91 chembio: 5 hits
API 8.92 chembio: 2 hits
API 8.93 chembio: No hits
API 8.94 chembio: 2 hits
API 8.9

API 9.155 hmdb: 13 hits
API 9.156 hmdb: No hits
API 9.157 hmdb: No hits
API 9.158 hmdb: No hits
API 9.159 hmdb: No hits
API 9.160 hmdb: No hits
API 9.161 hmdb: 5 hits
API 9.162 hmdb: No hits
API 9.163 hmdb: No hits
API 9.164 hmdb: No hits
API 9.165 hmdb: No hits
API 9.166 hmdb: 2 hits
API 9.167 hmdb: No hits
API 9.168 hmdb: No hits
API 9.169 hmdb: 2 hits
API 9.170 hmdb: No hits
API 9.171 hmdb: 4 hits
API 9.172 hmdb: 2 hits
API 9.173 hmdb: No hits
API 9.174 hmdb: No hits
API 9.175 hmdb: No hits
API 9.176 hmdb: No hits
API 9.177 hmdb: No hits
API 9.178 hmdb: No hits
API 9.179 hmdb: 4 hits
API 9.180 hmdb: No hits
API 9.181 hmdb: No hits
API 9.182 hmdb: No hits
API 9.183 hmdb: No hits
API 9.184 hmdb: No hits
API 9.185 hmdb: No hits
API 9.186 hmdb: No hits
API 9.187 hmdb: No hits
API 9.188 hmdb: No hits
API 9.189 hmdb: No hits
API 9.190 hmdb: No hits
API 9.191 hmdb: No hits
API 9.192 hmdb: No hits
API 9.193 hmdb: 2 hits
API 9.194 hmdb: No hits
API 9.195 hmdb: 5 hits
API 9.196 hmdb: No hits


## Step 3: Display and Filter results
This section demonstrates post-query filtering done in Python. Later, more advanced filtering functions will be added to the **query path execution** module for interleaved filtering, thereby enabling longer query paths. More details to come...

First, all matching paths can be exported to a data frame. Let's examine a sample of those results.

In [10]:
df = fc.display_table_view()

In [11]:
df

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,output_type,output_name,output_id
0,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,7841,HGNC:7841,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,CHEBI:48729,CHEBI:CHEBI:48729
1,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,19378,HGNC:19378,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,CHEBI:48729,CHEBI:CHEBI:48729
2,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,COX2,NCBIGene:4513,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,CHEBI:48729,CHEBI:CHEBI:48729
3,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,EPB42,NCBIGene:2038,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,CHEBI:48729,CHEBI:CHEBI:48729
4,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,RPS2,NCBIGene:6187,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,CHEBI:48729,CHEBI:CHEBI:48729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87690,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,MSC,NCBIGene:9242,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,2-OXO-4-METHYLPENTANOIC ACID,name:2-OXO-4-METHYLPENTANOIC ACID
87691,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,MSC,NCBIGene:9242,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,3-METHYLBUTANOIC ACID,name:3-METHYLBUTANOIC ACID
87692,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,MSC,NCBIGene:9242,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,3-METHYLHISTIDINE,name:3-METHYLHISTIDINE
87693,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,MSC,NCBIGene:9242,related_to,Translator Text Mining Provider,CORD Gene API,None,ChemicalSubstance,(+)-ENTEROLACTONE,name:(+)-ENTEROLACTONE


Next, let's examine which chemical compounds were most frequently mentioned.

In [28]:
df.output_name.value_counts().head(20)

.BETA.-D-GLUCOPYRANOSE         242
NITRIC OXIDE                   213
CALCIUM                        211
CHEBI:35222                    195
ALCOHOL                        182
CHEBI:24433                    177
PHARMACEUTICAL PREPARATIONS    170
INTERFERING RNA, SMALL         169
PHOSPHORYL                     159
CHEBI:59132                    158
CHOLESTEROL                    157
WATER                          156
DEXAMETHASONE                  155
CHEBI:18059                    151
3-METHYLADENINE DNA            150
DINOPROSTONE                   147
LEAD(0)                        144
CHEBI:35224                    144
DOXORUBICIN                    144
LIPOPOLYSACCHARIDES            142
Name: output_name, dtype: int64

List all unique predicates

In [16]:
df.pred1.unique()

array(['related_to', 'positively_regulates', 'negatively_regulates',
       'physically_interacts_with'], dtype=object)

In [17]:
df.pred2.unique()

array(['related_to', nan, 'physically_interacts_with', 'affects',
       'negatively_regulates', 'positively_regulates', 'metabolize',
       'disrupts'], dtype=object)

### Filter for drugs negatively regulates genes which are related to Endocytosis

In [23]:
df1 = df[(df["pred1"] == "related_to") & (df["pred2"] == "negatively_regulates")]

In [26]:
df1

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,output_type,output_name,output_id
6040,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,SNORA62,NCBIGene:6044,negatively_regulates,SEMMED,SEMMED Gene API,9060700,ChemicalSubstance,4-EPITETRACYCLINE,name:4-EPITETRACYCLINE
6050,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,TP53,NCBIGene:7157,negatively_regulates,SEMMED,SEMMED Gene API,9927213,ChemicalSubstance,4-EPITETRACYCLINE,name:4-EPITETRACYCLINE
6055,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,TNF,NCBIGene:7124,negatively_regulates,SEMMED,SEMMED Gene API,"16153435,16220550,8832479,9085232",ChemicalSubstance,4-EPITETRACYCLINE,name:4-EPITETRACYCLINE
6061,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,CA2,NCBIGene:760,negatively_regulates,SEMMED,SEMMED Gene API,106309,ChemicalSubstance,4-EPITETRACYCLINE,name:4-EPITETRACYCLINE
6067,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,CISH,NCBIGene:1154,negatively_regulates,SEMMED,SEMMED Gene API,28373878,ChemicalSubstance,4-EPITETRACYCLINE,name:4-EPITETRACYCLINE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87649,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,DLD,NCBIGene:1738,negatively_regulates,SEMMED,SEMMED Gene API,29017857,ChemicalSubstance,5-METHOXYINDOLE-2-CARBOXYLIC ACID,name:5-METHOXYINDOLE-2-CARBOXYLIC ACID
87651,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,DLD,NCBIGene:1738,negatively_regulates,SEMMED,SEMMED Gene API,23139597,ChemicalSubstance,ANGELI'S SALT,name:ANGELI'S SALT
87660,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,DLL1,NCBIGene:28514,negatively_regulates,SEMMED,SEMMED Gene API,24552522,ChemicalSubstance,METHYLNALOXONIUM IODIDE,name:METHYLNALOXONIUM IODIDE
87673,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,MIP,NCBIGene:4284,negatively_regulates,SEMMED,SEMMED Gene API,25324695,ChemicalSubstance,3-HYDROXY-3-METHYLGLUTARYL-COENZYME A,name:3-HYDROXY-3-METHYLGLUTARYL-COENZYME A


Next, let's examine which chemical compounds were most frequently mentioned.

In [24]:
df1.output_name.value_counts().head(20)

INTERFERING RNA, SMALL         73
PHARMACEUTICAL PREPARATIONS    49
C1328819                       43
NITRIC OXIDE                   36
AGONISTS                       36
OXYGEN SPECIES, REACTIVE       35
MESSENGER RNA                  35
LIPOPOLYSACCHARIDES            33
.BETA.-D-GLUCOPYRANOSE         32
ALCOHOL                        30
CIS-RESVERATROL                30
DEXAMETHASONE                  27
ANTISENSE OLIGONUCLEOTIDES     27
SIROLIMUS                      27
3-METHYLADENINE DNA            26
CURCUMIN                       25
GLUCOCORTICOID                 25
MICRORNA                       23
BINDING SITE                   23
SB 203580                      22
Name: output_name, dtype: int64

### Detailed literature support between connection of Endocytosis and DEXAMETHASONE

In [27]:
df[(df["pred1"] == "related_to") & (df["pred2"] == "negatively_regulates") & (df["output_name"] == "DEXAMETHASONE")]

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,output_type,output_name,output_id
15077,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,COX2,NCBIGene:4513,negatively_regulates,SEMMED,SEMMED Gene API,9249588,ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15082,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,EGF,NCBIGene:1950,negatively_regulates,SEMMED,SEMMED Gene API,9329825,ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15100,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,RHO,NCBIGene:6010,negatively_regulates,SEMMED,SEMMED Gene API,16424787,ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15103,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,SNORA62,NCBIGene:6044,negatively_regulates,SEMMED,SEMMED Gene API,17714695,ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15109,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,GPI,NCBIGene:2821,negatively_regulates,SEMMED,SEMMED Gene API,"10521370,10608467",ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15123,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,ABCB1,NCBIGene:5243,negatively_regulates,SEMMED,SEMMED Gene API,7654329,ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15128,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,IFNA1,NCBIGene:3439,negatively_regulates,SEMMED,SEMMED Gene API,"17418379,18242001,19019941,21832855,2453506,86...",ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15133,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,TP53,NCBIGene:7157,negatively_regulates,SEMMED,SEMMED Gene API,"11562347,15987750",ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15136,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,AKT1,NCBIGene:207,negatively_regulates,SEMMED,SEMMED Gene API,"15550386,18328801,20371624,24066853,24613413,2...",ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
15141,ENDOCYTOSIS,BiologicalProcess,related_to,Translator Text Mining Provider,CORD Biological Process API,None,Gene,TLR4,NCBIGene:7099,negatively_regulates,SEMMED,SEMMED Gene API,"18034897,28232829",ChemicalSubstance,DEXAMETHASONE,name:DEXAMETHASONE
